In [1]:
from sklearn.ensemble import RandomForestClassifier
import os
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import KFold
import csv
import pandas as pd

In [2]:
class RandomForest:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.rf = RandomForestClassifier(n_estimators=100, random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.rf.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.rf.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [5]:
column_headings = ["Morbidity Class", "RF_Macro F1", "RF_Micro F1"]

with open("./results/word-embeddings-features/performance_RF.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis = 1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        rf_obj = RandomForest(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        rf_obj.train()

        f1_macro, f1_micro = rf_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_RF.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_RF.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
(514, 100) (514,) (58, 100) (58,)
(514, 100) (514,) (58, 100) (58,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
(515, 100) (515,) (57, 100) (57,)
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(493, 100) (493,) (55, 100) (55,)
(494, 100) (494,) (54, 100) (54,)
(494, 100) (494,) (54, 100) (54,)
Macro F1 score: 0.7111840569539151 and Micro F1 Score 0.7317845117845118
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
(218, 10